In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [13]:
from plug_mongo import get_entries
df = pd.DataFrame(get_entries()).drop(["_id", "timestamp"], axis=1).set_index("game_id")
df.head()

,label,player_0_w,player_0_l,player_0_co_w,player_0_co_l,player_0_ct_wr,player_1_w,player_1_l,player_1_co_w,player_1_co_l,...,player_8_w,player_8_l,player_8_co_w,player_8_co_l,player_8_ct_wr,player_9_w,player_9_l,player_9_co_w,player_9_co_l,player_9_ct_wr
game_id,,,,,,,,,,,,,,,,,,,,,
548809662,0,216,181,145,91,0.5354,174,147,8,3,...,297,284,1,0,0.5003,941,930,43,35,0.5093
4484824702,0,541,503,27,28,0.5161,160,111,0,0,...,269,202,9,3,0.4971,988,954,65,78,0.4948
548810655,1,155,146,20,6,0.4879,55,46,0,0,...,186,144,22,23,0.5088,55,45,0,1,0.5229
6140028214,1,686,645,43,41,0.5082,368,281,17,11,...,234,198,25,32,0.5197,2090,2035,390,340,0.5306
1351099000,1,31,14,23,8,0.5043,38,12,2,0,...,17,6,8,3,0.5148,125,103,0,0,0.4933


In [15]:
df.isnull().sum().sum()

0

In [4]:
def add_one(column):
	return df[column].apply(lambda x: x+1)

def get_winrate(column):
	win = add_one(column+'_w')
	los = add_one(column+'_l')
	return (win / (win + los)).rename(column+"_wr")

get_winrate('player_0').head()

game_id
548809662     0.543860
4484824702    0.518164
548810655     0.514851
6140028214    0.515379
1351099000    0.680851
Name: player_0_wr, dtype: float64

In [5]:
result = [df]
result.extend([get_winrate(f"player_{x}") for x in range(10)])
result.extend([get_winrate(f"player_{x}_co") for x in range(10)])
df_winrates = pd.concat(result, axis=1)
df_winrates.head()

,label,player_0_w,player_0_l,player_0_co_w,player_0_co_l,player_0_ct_wr,player_1_w,player_1_l,player_1_co_w,player_1_co_l,...,player_0_co_wr,player_1_co_wr,player_2_co_wr,player_3_co_wr,player_4_co_wr,player_5_co_wr,player_6_co_wr,player_7_co_wr,player_8_co_wr,player_9_co_wr
game_id,,,,,,,,,,,,,,,,,,,,,
548809662,0,216,181,145,91,0.5354,174,147,8,3,...,0.613445,0.692308,0.789474,0.482759,0.700000,0.500000,0.584211,0.666667,0.666667,0.550000
4484824702,0,541,503,27,28,0.5161,160,111,0,0,...,0.491228,0.500000,0.533537,0.727273,0.545763,0.428571,0.666667,0.549020,0.714286,0.455172
548810655,1,155,146,20,6,0.4879,55,46,0,0,...,0.750000,0.500000,0.500000,0.285714,0.333333,0.500000,0.500000,0.500000,0.489362,0.333333
6140028214,1,686,645,43,41,0.5082,368,281,17,11,...,0.511628,0.600000,0.764706,0.625000,0.800000,0.642857,0.527538,0.606383,0.440678,0.534153
1351099000,1,31,14,23,8,0.5043,38,12,2,0,...,0.727273,0.750000,0.428571,0.500000,0.666667,0.531469,0.666667,0.500000,0.692308,0.500000


In [16]:
df_winrates.isnull().sum().sum()

0

In [6]:
train_dataset = df_winrates.sample(frac=0.8, random_state=0)
test_dataset = df_winrates.drop(train_dataset.index)
train_dataset.head()

,label,player_0_w,player_0_l,player_0_co_w,player_0_co_l,player_0_ct_wr,player_1_w,player_1_l,player_1_co_w,player_1_co_l,...,player_0_co_wr,player_1_co_wr,player_2_co_wr,player_3_co_wr,player_4_co_wr,player_5_co_wr,player_6_co_wr,player_7_co_wr,player_8_co_wr,player_9_co_wr
game_id,,,,,,,,,,,,,,,,,,,,,
2628711553,1,458,456,2,1,0.532700,0,0,0,0,...,0.600000,0.500000,0.666667,0.333333,0.545455,0.428571,0.443182,0.568306,0.400000,0.750000
6227393249,0,20,22,0,0,0.500100,56,56,8,1,...,0.500000,0.818182,0.500000,0.500000,0.500000,0.666667,0.437500,0.500000,0.500000,0.500000
4490874955,1,658,641,645,603,0.452300,139,134,3,1,...,0.516800,0.666667,0.425000,0.666667,0.750000,0.483871,0.695652,0.857143,0.558140,0.428571
1241429289,0,76,80,0,0,0.500100,229,217,0,1,...,0.500000,0.333333,0.250000,0.666667,0.684211,0.642857,0.818182,0.250000,0.785714,0.333333
1242270360,0,138,149,1,0,0.477387,57,66,25,32,...,0.666667,0.440678,0.666667,0.652174,0.250000,0.600000,0.500000,0.571429,0.533333,0.666667


In [7]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('label')
test_labels = test_features.pop('label')
train_labels.head()

game_id
2628711553    1
6227393249    0
4490874955    1
1241429289    0
1242270360    0
Name: label, dtype: int64

In [8]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(train_features)

In [9]:
def build_and_compile_model(norm, width, depth, activation='relu'):
    layer_l = [norm]
    layer_l.extend([layers.Dense(width, activation=activation) for _ in range(depth)])
    layer_l.append(layers.Dense(1, activation='softmax'))
    model = keras.Sequential(layer_l)
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.001))
    return model

In [10]:
dnn_model = build_and_compile_model(normalizer, 64, 2)
dnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 70)               141       
 n)                                                              
                                                                 
 dense (Dense)               (None, 64)                4544      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 8,910
Trainable params: 8,769
Non-trainable params: 141
_________________________________________________________________


In [11]:
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=1, epochs=100)

Epoch 1/100
393/393 [==============================] - 3s 5ms/step - loss: 0.4785 - val_loss: 0.4893
Epoch 2/100
393/393 [==============================] - 2s 5ms/step - loss: 0.4785 - val_loss: 0.4893
Epoch 3/100
393/393 [==============================] - 2s 5ms/step - loss: 0.4785 - val_loss: 0.4893
Epoch 4/100
393/393 [==============================] - 2s 5ms/step - loss: 0.4785 - val_loss: 0.4893
Epoch 5/100
393/393 [==============================] - 2s 5ms/step - loss: 0.4785 - val_loss: 0.4893
Epoch 6/100
393/393 [==============================] - 3s 6ms/step - loss: 0.4785 - val_loss: 0.4893
Epoch 7/100
393/393 [==============================] - 2s 5ms/step - loss: 0.4785 - val_loss: 0.4893
Epoch 8/100
393/393 [==============================] - 2s 5ms/step - loss: 0.4785 - val_loss: 0.4893
Epoch 9/100
393/393 [==============================] - 2s 6ms/step - loss: 0.4785 - val_loss: 0.4893
Epoch 10/100
393/393 [==============================] - 2s 5ms/step - loss: 0.4785 - val_lo

KeyboardInterrupt: 

In [12]:
early_stop = keras.callbacks.EarlyStopping(patience=5, verbose=0, restore_best_weights=True)
activations = ['relu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'tanh', 'selu', 'elu']

def test_model(width, depth, activation, verbose=1):
    with tf.device("GPU:0"):
        model = build_and_compile_model(normalizer, width, depth, activation)
        history_return = model.fit(
            train_features,
            train_labels,
            validation_split=0.2,
            verbose=verbose, epochs=100,
            callbacks = [early_stop]
        )
    return min(history_return.history['val_loss']), len(history_return.history['val_loss']), model

In [60]:
[(act, test_model(18, 4, act, 0)) for act in activations]

[('relu',
  (0.4873979687690735,
   8,
   <keras.engine.sequential.Sequential at 0x1dc18f95748>)),
 ('sigmoid',
  (0.45493367314338684,
   23,
   <keras.engine.sequential.Sequential at 0x1da6f8999c8>)),
 ('softmax',
  (0.452161967754364,
   23,
   <keras.engine.sequential.Sequential at 0x1dc0805f0c8>)),
 ('softplus',
  (0.45976924896240234,
   24,
   <keras.engine.sequential.Sequential at 0x1da3e0d1488>)),
 ('softsign',
  (0.4557131826877594,
   16,
   <keras.engine.sequential.Sequential at 0x1dc27d1c348>)),
 ('tanh',
  (0.46087461709976196,
   22,
   <keras.engine.sequential.Sequential at 0x1da6fae0288>)),
 ('selu',
  (0.4837348461151123,
   18,
   <keras.engine.sequential.Sequential at 0x1da3df6b3c8>)),
 ('elu',
  (0.4750992953777313,
   10,
   <keras.engine.sequential.Sequential at 0x1dc07e7e708>)),
 ('exponential',
  (nan, 5, <keras.engine.sequential.Sequential at 0x1dc07a9f548>))]

In [13]:
modes = []
for x in [18, 35, 70, 140]:
    for y in [1, 2, 3, 4, 5, 6]:
        for z in ['softmax', 'sigmoid']:
            modes.append((x, y, z))
results = [(mode, test_model(mode[0], mode[1], mode[2], 1)) for mode in modes]
results

Epoch 1/100
392/392 [==============================] - 3s 5ms/step - loss: 0.4784 - val_loss: 0.4872
Epoch 2/100
392/392 [==============================] - 2s 5ms/step - loss: 0.4784 - val_loss: 0.4872
Epoch 3/100
392/392 [==============================] - 2s 5ms/step - loss: 0.4784 - val_loss: 0.4872
Epoch 4/100
392/392 [==============================] - 2s 5ms/step - loss: 0.4784 - val_loss: 0.4872
Epoch 5/100
 14/392 [>.............................] - ETA: 1s - loss: 0.5312

KeyboardInterrupt: 

In [ ]:
[(x[0], x[1][0]) for x in sorted(results, key=lambda x: x[1][0])]

In [14]:
m = test_model(288, 2, 'softmax')
m

Epoch 1/100
392/392 [==============================] - 3s 6ms/step - loss: 0.4784 - val_loss: 0.4872
Epoch 2/100
392/392 [==============================] - 2s 6ms/step - loss: 0.4784 - val_loss: 0.4872
Epoch 3/100
392/392 [==============================] - 2s 6ms/step - loss: 0.4784 - val_loss: 0.4872
Epoch 4/100
392/392 [==============================] - 2s 6ms/step - loss: 0.4784 - val_loss: 0.4872
Epoch 5/100
392/392 [==============================] - 2s 6ms/step - loss: 0.4784 - val_loss: 0.4872
Epoch 6/100
392/392 [==============================] - 2s 6ms/step - loss: 0.4784 - val_loss: 0.4872


(0.4872449040412903, 6, <keras.engine.sequential.Sequential at 0x20b11e66148>)

In [16]:
m[2].summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 70)               141       
 n)                                                              
                                                                 
 dense_2 (Dense)             (None, 288)               20448     
                                                                 
 dense_3 (Dense)             (None, 288)               83232     
                                                                 
 dense_4 (Dense)             (None, 1)                 289       
                                                                 
Total params: 104,110
Trainable params: 103,969
Non-trainable params: 141
_________________________________________________________________
